In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

In [2]:
crsp = pd.read_csv(r'C:\Users\edgar\Downloads\crsp_top1000.csv', sep='\t')
FF = pd.read_csv(r'C:\Users\edgar\Downloads\FF.csv', sep='\t')
ratios = pd.read_csv(r'C:\Users\edgar\Downloads\financialratios.csv', sep='\t')
Ratios_Manual = pd.read_excel(r'C:\Users\edgar\Downloads\FinancialRatiosManual.xlsx')

In [3]:
FF['date'] = pd.to_datetime(FF['date'])
Ratios_Manual['Financial Ratio'] = Ratios_Manual['Financial Ratio'].str.replace('\n',' ')
crsp['date'] = pd.to_datetime(crsp['date'])
ratios['public_date'] = pd.to_datetime(ratios['public_date'])
df_all = pd.merge_asof(ratios.sort_values('public_date'), crsp.sort_values('date'), 
                           by='permno', 
                           left_on='public_date', 
                           right_on='date',
                           tolerance=pd.Timedelta(3, 'days'))
df_all = df_all.sort_values(by = ['permno', 'date'])

In [4]:
df_shift = df_all[['permno','date','month','ret']]
L = []
for i in range(1,len(df_shift)) :
    if df_shift.iloc[i, 0] == df_shift.iloc[i-1, 0] :
        if (df_shift.iloc[i, 2] == df_shift.iloc[i-1, 2] + 1) or (df_shift.iloc[i, 2] == 1 and df_shift.iloc[i-1, 2] == 12) :
            L.append(df_shift.iloc[i, 3])
        else :
            L.append(np.nan)
    else :
        L.append(np.nan)

L = L + [np.nan]

df_all['Ret t+1'] = L

In [5]:
df_all_insample = df_all[df_all['public_date'] < '2017-01-01'].dropna(subset = ['prc'])
df_all_outsample = df_all[df_all['public_date'] >= '2017-01-01'].dropna(subset = ['prc'])

In [6]:
def Listret(fratio,df,quant) :
    Date = list(df.sort_values('date')['date'].drop_duplicates())
    ListRet = []
    for i in range(len(Date)-1) :
        test = df[df["date"] == Date[i]][['permno',fratio]].sort_values(fratio).dropna()
        testbottom20 = test[test[fratio] < test.quantile(quant)[fratio]]
        testtop20 = test[test[fratio] > test.quantile(1-quant)[fratio]]
        df_all_test_top = df[df["date"] == Date[i+1]][df["permno"].isin(list(testtop20['permno']))]
        df_all_test_bottom = df[df["date"] == Date[i+1]][df["permno"].isin(list(testbottom20['permno']))]
        ListRet.append((sum([float(x) for x in list(df_all_test_top['ret'])]) - sum([float(x) for x in list(df_all_test_bottom['ret'])]))/(len(df_all_test_top) + len(df_all_test_bottom))) 
    return(ListRet)

In [7]:
def ListretMultiple(List_fratio,df,quant) :
    Date = list(df.sort_values('date')['date'].drop_duplicates())
    ListRet = []
    for i in range(len(Date)-1) :
        ListFirmTop = []
        ListFirmBottom = []
        for ratio in List_fratio :
            test = df[df["date"] == Date[i]][['permno',ratio]].sort_values(ratio).dropna()
            testbottom = test[test[ratio] < test.quantile(quant)[ratio]]
            testtop = test[test[ratio] > test.quantile(1-quant)[ratio]]
            ListFirmBottom += list(testbottom['permno'])
            ListFirmTop += list(testtop['permno'])
        df_all_test_top = df[df["date"] == Date[i+1]][df["permno"].isin(ListFirmTop)]
        df_all_test_bottom = df[df["date"] == Date[i+1]][df["permno"].isin(ListFirmBottom)]
        ListRet.append((sum([float(x) for x in list(df_all_test_top['ret'])]) - sum([float(x) for x in list(df_all_test_bottom['ret'])]))/(len(df_all_test_top) + len(df_all_test_bottom))) 
    return(ListRet)

In [22]:
def Listret_Industry(fratio,df,quant) :
    Date = list(df.sort_values('date')['date'].drop_duplicates())
    ListRet = []
    for i in range(len(Date)-1) :
        test = df[df["date"] == Date[i]][['permno','sic2',fratio]].sort_values(fratio).dropna()
        List_Industry = list(test['sic2'].drop_duplicates())
        List_Long = []
        List_Short = []
        for ind in List_Industry :
            testind = test[test['sic2'] == ind]
            testbottom = testind[testind[fratio] < testind.quantile(quant)[fratio]]
            List_Short += list(testbottom['permno'].drop_duplicates())
            testtop = testind[testind[fratio] > testind.quantile(1-quant)[fratio]]
            List_Long += list(testtop['permno'].drop_duplicates())
        df_all_test_top = df[df["date"] == Date[i+1]][df["permno"].isin(List_Long)]
        df_all_test_bottom = df[df["date"] == Date[i+1]][df["permno"].isin(List_Short)]
        ListRet.append((sum([float(x) for x in list(df_all_test_top['ret'])]) - sum([float(x) for x in list(df_all_test_bottom['ret'])]))/(len(df_all_test_top) + len(df_all_test_bottom))) 
    return(ListRet)

In [23]:
x = 'roa'
quant = 0.05
ListIndustry1 = Listret_Industry(x,df1,quant)
ListIndustry2 = Listret_Industry(x,df2,quant)
print(sum(ListIndustry1))
print(sum(ListIndustry2))
print(np.var(ListIndustry1))
print(np.var(ListIndustry2))
df_reg = Regress(df2,ListIndustry2)
print(sm.ols(formula="R_Rf ~ mktrf+smb+hml+qual+umd", data=df_reg).fit().params[0])
print(np.sqrt(12)*df_reg['R_Rf'].mean()/df_reg['R_Rf'].std())

0.5255918807475067
0.052273641179532634
0.00029645117980497477
0.00019118831910235312
-0.002635213215314843
-0.04741259982647219


In [9]:
def Regress(df,ListRet) :
    df_regress = pd.DataFrame({'date' : list(df.sort_values('date')['date'].drop_duplicates())[1:], 'Return Portfolio' : ListRet}).merge(FF, how='left', on='date')
    df_regress['R_Rf'] = df_regress['Return Portfolio'] - df_regress['rf']  
    return(df_regress)

In [10]:
def DF_Recap_Sum(x,df,quant) :
    try: result = sum(Listret(x,df,quant)) 
    except: result =  np.nan
    return result

def DF_Recap_Sum_Multiple(x,df,quant) :
    try: result = sum(ListretMultiple(x,df,quant)) 
    except: result =  np.nan
    return result

def DF_Recap_Var(x,df,quant) :
    try: result = np.var(Listret(x,df,quant)) 
    except: result =  np.nan
    return result

def DF_Recap_Var_Multiple(x,df,quant) :
    try: result = np.var(ListretMultiple(x,df,quant)) 
    except: result =  np.nan
    return result

def DF_Recap_Alpha(x,df,quant) :
    try: 
        df_reg = Regress(df,Listret(x,df,quant))
        regress = sm.ols(formula="R_Rf ~ mktrf+smb+hml+qual+umd", data=df_reg).fit()
        result = regress.params[0]
    except: result =  np.nan
    return result

def DF_Recap_Sharpe(x,df,quant) :
    try: 
        df_reg = Regress(df,Listret(x,df,quant))
        result = np.sqrt(12)*df_reg['R_Rf'].mean()/df_reg['R_Rf'].std()
    except: result =  np.nan
    return result

def DF_Recap_Alpha_Multiple(x,df,quant) :
    try: 
        df_reg = Regress(df,ListretMultiple(x,df,quant))
        regress = sm.ols(formula="R_Rf ~ mktrf+smb+hml+qual+umd", data=df_reg).fit()
        result = regress.params[0]
    except: result =  np.nan
    return result

def DF_Recap_Sharpe_Multiple(x,df,quant) :
    try: 
        df_reg = Regress(df,ListretMultiple(x,df,quant))
        result = np.sqrt(12)*df_reg['R_Rf'].mean()/df_reg['R_Rf'].std()
    except: result =  np.nan
    return result

In [11]:
df1 = df_all_insample
df2 = df_all_outsample
quant = 0.05

In [12]:
Recap = pd.DataFrame({'Ratios' : list(Ratios_Manual['Variable Name'])})
Recap['Return In Sample'] = Recap['Ratios'].apply(lambda x: DF_Recap_Sum(x,df1,quant))
Recap['Return Out Sample'] = Recap['Ratios'].apply(lambda x: DF_Recap_Sum(x,df2,quant))
Recap['Variance In Sample'] = Recap['Ratios'].apply(lambda x: DF_Recap_Var(x,df1,quant))
Recap['Variance Out Sample'] = Recap['Ratios'].apply(lambda x: DF_Recap_Var(x,df2,quant))
Recap['Sharpe Ratio'] = Recap['Ratios'].apply(lambda x: DF_Recap_Sharpe(x,df2,quant))
Recap['Alpha'] = Recap['Ratios'].apply(lambda x: DF_Recap_Alpha(x,df2,quant))
Recap

,Ratios,Return In Sample,Return Out Sample,Variance In Sample,Variance Out Sample,Sharpe Ratio,Alpha
0,capital_ratio,3.189712,0.211564,0.002767,0.000673,0.271081,-0.000583
1,equity_invcap,-2.247823,-0.014776,0.002686,0.000530,-0.170623,0.000849
2,debt_invcap,3.112235,0.135120,0.002999,0.000709,0.125616,-0.001664
3,totdebt_invcap,1.835225,0.031683,0.002199,0.000489,-0.074121,-0.001565
4,at_turn,1.136155,0.230045,0.000640,0.000518,0.350487,-0.001077
...,...,...,...,...,...,...,...
67,pe_inc,0.157866,0.018301,0.000219,0.000159,-0.180322,-0.000676
68,pe_op_basic,-0.289427,0.031500,0.000287,0.000215,-0.112809,-0.000662
69,pe_op_dil,NaN,0.000304,NaN,0.000216,-0.214931,-0.001118
70,ps,-1.221323,-0.342099,0.001857,0.001619,-0.493053,-0.004843


In [13]:
PC1 = ['rd_sale','npm']

PC2 = ['aftret_invcapx'] 

PC3 = ['rd_sale' ,'npm' ,'staff_sale']  

PC4 = ['bm', 'npm', 'gpm', 'roe', 'aftret_eq', 'cash_lt', 'cash_debt', 'dltt_be', 'sale_invcap', 'rd_sale', 'staff_sale', 'year']

PC5 = ['bm', 'aftret_eq', 'capital_ratio', 'dltt_be', 'debt_capital', 'accrual', 'ptb']

In [14]:
Recap_PCA = pd.DataFrame({'Ratios' : ["PC" +str(i) for i in range(1,6)]})
Recap_PCA['Return In Sample'] = Recap_PCA['Ratios'].apply(lambda x: DF_Recap_Sum_Multiple(globals()[x],df1,quant))
Recap_PCA['Return Out Sample'] = Recap_PCA['Ratios'].apply(lambda x: DF_Recap_Sum_Multiple(globals()[x],df2,quant))
Recap_PCA['Variance In Sample'] = Recap_PCA['Ratios'].apply(lambda x: DF_Recap_Var_Multiple(globals()[x],df1,quant))
Recap_PCA['Variance Out Sample'] = Recap_PCA['Ratios'].apply(lambda x: DF_Recap_Var_Multiple(globals()[x],df2,quant))
Recap_PCA['Sharpe Ratio'] = Recap_PCA['Ratios'].apply(lambda x: DF_Recap_Sharpe_Multiple(globals()[x],df2,quant))
Recap_PCA['Alpha'] = Recap_PCA['Ratios'].apply(lambda x: DF_Recap_Alpha_Multiple(globals()[x],df2,quant))
Recap_PCA

,Ratios,Return In Sample,Return Out Sample,Variance In Sample,Variance Out Sample,Sharpe Ratio,Alpha
0,PC1,1.795365,0.289932,0.000447,0.000273,0.653888,0.000248
1,PC2,1.420810,0.367216,0.001278,0.000814,0.512750,0.000531
2,PC3,2.135228,0.386955,0.000626,0.000634,0.614676,0.000584
3,PC4,0.998010,0.112001,0.000112,0.000091,0.228494,-0.001006
4,PC5,0.249576,0.057802,0.000119,0.000046,-0.055529,-0.001015


In [24]:
Recap_PCA.to_excel('Recap_PCA.xlsx')

In [15]:
Recap_Pos = Recap.sort_values('Return In Sample',ascending=False).head(10)

In [16]:
Recap_Pos

,Ratios,Return In Sample,Return Out Sample,Variance In Sample,Variance Out Sample,Sharpe Ratio,Alpha
0,capital_ratio,3.189712,0.211564,0.002767,0.000673,0.271081,-0.000583
34,staff_sale,3.178321,0.689086,0.002785,0.003966,0.478274,0.001527
11,invt_act,3.138706,0.784493,0.002704,0.004230,0.533934,-0.000356
2,debt_invcap,3.112235,0.135120,0.002999,0.000709,0.125616,-0.001664
18,short_debt,2.769071,0.040124,0.002806,0.001435,-0.032849,-0.000801
22,dltt_be,2.692490,0.157275,0.003517,0.001122,0.131866,-0.001685
57,dpr,2.650512,0.762252,0.002196,0.003403,0.576782,0.004024
33,adv_sale,2.488828,0.326920,0.003098,0.003495,0.213813,-0.002390
9,sale_invcap,1.888746,0.521820,0.001057,0.000344,1.189280,0.003446
3,totdebt_invcap,1.835225,0.031683,0.002199,0.000489,-0.074121,-0.001565


In [17]:
def Best_Group(ratio) :
    ListRatios = list(Recap_Pos[Recap_Pos["Ratios"] != ratio]["Ratios"])
    ListSelect = [ratio]
    ListRet0 = ListretMultiple(ListSelect,df1,quant)
    A = 5
    U = np.mean(ListRet0) -1/2*A*np.var(ListRet0)
    for x in ListRatios[1:] :
        ListTest = ListSelect + [x]
        ListRet1 = ListretMultiple(ListSelect,df1,quant)
        UTest = np.mean(ListRet1) -1/2*A*np.var(ListRet1)
        if UTest > U :
            ListSelect = ListTest
            U = UTest
    return(ListSelect,U)

In [18]:
ListBestGroup = [Best_Group(x) for x in list(Recap_Pos["Ratios"])]

In [73]:
pd.DataFrame([(ListBestGroup[i][0][0],ListBestGroup[i][1]) for i in range(len(ListBestGroup))]).to_excel('BestGroup.xlsx')